In [17]:
print("✅ everything is working")


✅ everything is working


In [18]:
import numpy as np
import pandas as pd

print("Imports OK ")


Imports OK 


In [19]:
# Load cleaned unsupervised dataset
df_unsup = pd.read_csv("../Section 1/clean_unsupervised.csv")

print("Number of comments:", len(df_unsup))
df_unsup.head()


Number of comments: 1107946


,author,subreddit,created_utc,body,body_normalized,body_clean
0,ejchristian86,TwoXChromosomes,1.388534e+09,I hadn't ever heard of them before joining thi...,i hadn't ever heard of them before joining thi...,hear join subreddit big thing apparently commo...
1,ZenDragon,gaming,1.388534e+09,"At 7680 by 4320 with 64x AA, right?","at 7680 by 4320 with 64x aa, right?",7680 4320 64x aa right
2,savoytruffle,AskReddit,1.388534e+09,bite me,bite me,bite
3,hentercenter,stlouisblues,1.388534e+09,Damn that was a good penalty :(,damn that was a good penalty :(,damn good penalty
4,rick-o-suave,army,1.388534e+09,I swore into DEP on 6-OCT and I left 5-NOV und...,i swore into dep on 6-oct and i left 5-nov und...,swear dep 6 oct leave 5 nov 18x contract month...


In [20]:
df_unsup['body_clean'].str.len().describe()


count    1.092674e+06
mean     8.088918e+01
std      1.415220e+02
min      1.000000e+00
25%      2.300000e+01
50%      5.700000e+01
75%      7.500000e+01
max      1.079900e+04
Name: body_clean, dtype: float64

In [21]:
df_sample = df_unsup.sample(
    n=100_000,
    random_state=42
)

# Clean missing or empty documents
df_sample = df_sample.dropna(subset=['body_clean'])
df_sample = df_sample[df_sample['body_clean'].str.strip() != ""]
df_sample['body_clean'] = df_sample['body_clean'].astype(str)


In [22]:
len(df_sample)


98597

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [24]:

tfidf = TfidfVectorizer(
    max_features=10000,
    min_df=5,
    max_df=0.5,
    stop_words='english'
)

X = tfidf.fit_transform(df_sample['body_clean'])
X.shape


(98597, 10000)

In [25]:
from sklearn.decomposition import TruncatedSVD

n_components = 50
svd = TruncatedSVD(n_components=n_components, random_state=42)
X_reduced = svd.fit_transform(X)
X_reduced.shape


(98597, 50)

In [26]:
from sklearn.cluster import KMeans

k = 10
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_reduced)

labels


array([5, 1, 6, ..., 6, 6, 6], dtype=int32)

In [27]:
df_sample = df_sample.reset_index(drop=True)
df_sample["cluster"] = labels
df_sample.head()


,author,subreddit,created_utc,body,body_normalized,body_clean,cluster
0,Lokikong,IAmA,1.388696e+09,"I think you mean ""*Pass* the pipe.""","i think you mean ""*pass* the pipe.""",think mean pass pipe,5
1,MrN4T3,projectcar,1.390693e+09,No I've just put so much work into it trying t...,no i've just put so much work into it trying t...,work try ready event happen lose expect 50 dol...,1
2,T3chnopsycho,explainlikeimfive,1.389039e+09,Sweden is not Switzerland ;) \n\nOtherwise I a...,sweden is not switzerland ;) otherwise i agree...,sweden switzerland agree like user post switze...,6
3,1EyedPokerface,AskReddit,1.389594e+09,ah I will sleep better after reading that,ah i will sleep better after reading that,ah sleep well read,6
4,Not_a_ZED,pcmasterrace,1.389288e+09,Yep. Time to upgrade to a monitor mount.,yep. time to upgrade to a monitor mount.,yep time upgrade monitor mount,6


In [28]:
df_sample["cluster"].value_counts().sort_index()


cluster
0     6008
1    10726
2     9050
3     3416
4     1865
5     7247
6    54955
7     1175
8     3106
9     1049
Name: count, dtype: int64

In [29]:
df_sample[df_sample["cluster"] == 0]["body"].head(15)


6      :( I'm sorry they rejected it. I guess it is d...
16     &gt; I'm the Al Queda of the anti gun movement...
20     The scent of bourjois is pretty strong and if ...
37     Looks like this got caught in the spam filter....
66     That's actually where my husband got the name ...
67     Do you ever do a reveal at the end of the conv...
93     "Go fuck yourself, poor people!"\n\nThis will ...
97     This is rather sad because it is true. I would...
120    How about the pointed pronunciation of the let...
135    EXACTLY. and i don't like how jerky their move...
136    It looks like Boone is just beating the shit o...
140    Sourced here: https://play.google.com/store/ap...
159    Looks like a fence because the dog looks like ...
171    Yeah, there are a couple of different mechanis...
201    Shit I can't like this more than once. I'd giv...
Name: body, dtype: object

In [30]:
df_sample[df_sample["cluster"] == 1]["body"].head(15)


1      No I've just put so much work into it trying t...
13     No I've just put so much work into it trying t...
30     No I've just put so much work into it trying t...
32     No I've just put so much work into it trying t...
45     No I've just put so much work into it trying t...
52     No I've just put so much work into it trying t...
59     No I've just put so much work into it trying t...
75     No I've just put so much work into it trying t...
88     No I've just put so much work into it trying t...
92     No I've just put so much work into it trying t...
95     No I've just put so much work into it trying t...
101    No I've just put so much work into it trying t...
103    No I've just put so much work into it trying t...
106    No I've just put so much work into it trying t...
128    No I've just put so much work into it trying t...
Name: body, dtype: object

In [31]:
df_sample[df_sample["cluster"] == 2]["body"].head(15)


5      Glad it went fairly smoothly. I was a bridesma...
8      Glad it went fairly smoothly. I was a bridesma...
10     Glad it went fairly smoothly. I was a bridesma...
15     Glad it went fairly smoothly. I was a bridesma...
23     Glad it went fairly smoothly. I was a bridesma...
40     Glad it went fairly smoothly. I was a bridesma...
41     Glad it went fairly smoothly. I was a bridesma...
63     Glad it went fairly smoothly. I was a bridesma...
102    Glad it went fairly smoothly. I was a bridesma...
112    Glad it went fairly smoothly. I was a bridesma...
113    Glad it went fairly smoothly. I was a bridesma...
124    Glad it went fairly smoothly. I was a bridesma...
130    Glad it went fairly smoothly. I was a bridesma...
153    Glad it went fairly smoothly. I was a bridesma...
157    Glad it went fairly smoothly. I was a bridesma...
Name: body, dtype: object

In [32]:
df_sample[df_sample["cluster"] == 3]["body"].head(15)


19     Though not a yank 
21     Though not a yank 
34     Though not a yank 
42     Though not a yank 
94     Though not a yank 
114    Though not a yank 
164    Though not a yank 
168    Though not a yank 
173    Though not a yank 
182    Though not a yank 
194    Though not a yank 
210    Though not a yank 
221    Though not a yank 
222    Though not a yank 
248    Though not a yank 
Name: body, dtype: object

In [33]:
df_sample[df_sample["cluster"] == 4]["body"].head(15)


29     Yep I sure do game. Been a gamer since I was m...
58      I'm 95% sure I saw this picture on my newsfeed. 
100    Once again. Thanks for the input. I'm going to...
117                                 &gt; Not sure \n\n:)
237        That's pretty much my reaction when losing...
239    Eh, Kiko Alonso was pretty beastly.  He plays ...
244    I thought maybe Karo light corn syrup was what...
253    If you're looking at it from a B movie perspec...
312    Because I don't need to be. I'm fine with the ...
369    Pretty much never. It's just not something I n...
503    i've missed last call for sure, though there i...
523    Thin knee high socks should do the trick! They...
537    I warned all the younger women there (all fami...
596    Im not sure of anything. I posted pictures... ...
625    Pretty sure that Bodies by Drowning Pool has n...
Name: body, dtype: object

In [34]:
df_sample[df_sample["cluster"] == 5]["body"].head(15)


0                    I think you mean "*Pass* the pipe."
7      I know we shouldn't trust Twitter but I've hea...
35      Seriously, just call them southwest. There's ...
78     I prefer the more upbeat version from *Going P...
80     I suppose it's possible. I've never had that h...
81     Thank you! The thing I need clarifying is: how...
89     &gt;embrace the grind, lower your shoulder and...
105    I know a few people that enjoy yoga but don't ...
108    There have been WAY too many of these ads post...
138    Exactly. That's why I wouldn't feel awkward be...
186    This is really the wrong sub in which to attem...
202    I see it as a way of donating to a website I u...
228           If only we could all think for ourselves! 
235    Untrue. I've done it on numerous occasions. Lo...
243    Ah... yeah.... I think I am kind of average on...
Name: body, dtype: object

In [35]:
df_sample[df_sample["cluster"] == 6]["body"].head(15)


2     Sweden is not Switzerland ;) \n\nOtherwise I a...
3             ah I will sleep better after reading that
4              Yep. Time to upgrade to a monitor mount.
9                    how are you going to learn though?
11    [Nom Nom Nom](http://www.youtube.com/watch?v=m...
12    Would you have someone shuffle the deck one wa...
14    While I don't believe in the feminist idea of ...
17    So I was picking you up in 30 minutes. You loo...
18       You are not a virgin in any sense of the word.
22    As long as they don't take our nieces, they ca...
24                      You walked out of the theatre? 
25    Okay, you're right. 95% of what she does, the ...
26    I never said you should buy.  I said the cheap...
27                                 Bottle up economics.
28                             Nite nite stalwart shibe
Name: body, dtype: object

In [36]:
df_sample[df_sample["cluster"] == 7]["body"].head(15)


91                Thanks. :p Finally someone understands!
141                                          Aww, thanks!
209     Wow. I was not expecting that. Thanks man! I'l...
220            Thank you for keeping me in your thoughts 
372     LOL. I can see why that could be confusing. \n...
448                                      Ok cool, thanks!
571                this is a really neat site thanks :)))
649     This is absolutely amazing, I can't believe th...
740                                                Thanks
839                               lol, its okay.  Thanks.
853     Thank you! \n\n[Wohoo!](https://24.media.tumbl...
1005                                              Thanks!
1038                                              Thanks!
1053                                   Well damn. Thanks!
1275    He's not on sleep meds. That was useful info t...
Name: body, dtype: object

In [37]:
df_sample[df_sample["cluster"] == 8]["body"].head(15)


50     I have no idea man. Best bet is to google. I h...
147                         Zotac is *clearly* the best.
149    Doesn't Harry just have some of the traits of ...
151                                           Good luck!
175    &gt;How do you keep from going crazy during th...
176    I like Crutchfield, and I'm sure other people ...
190    That's a good comparison, there have to be sta...
259    It was kind of lame when his girlfriend was An...
394       Best ride at the garage, aside from your mom. 
472                               poolerino is good too.
510    the part i love the best is where he saves up ...
512                                       u looking good
528    Fell for her in middle school, never let go. T...
589    Gosh, I always forget how long he's been broad...
637    Have you perchance played The Stanley Parable?...
Name: body, dtype: object

In [38]:
df_sample[df_sample["cluster"] == 9]["body"].head(15)


46      What the fuck? I don't see how people are okay...
373     That isn't true everywhere, and "most of the w...
377     more importantly, have you seen [the video](ht...
529     I enjoy both a lot! It's not something I learn...
803                                  ''Hey, wanna fuck?''
985                         This fight is classy as fuck.
994     But notice that the Japanese don't give a fuck...
1152    I'm actually happy to go back...\n\nI'm bored ...
1327                                        Fuck I'm old 
1391    Wait, a dude called her a "stripper whore" and...
1404                            Fuck no. Are you insane!?
1525    Fuck SeaWorld. I was just in Orlando and I fli...
1721    what the flying fuck is that i nthe background...
1827                                          Holey fuck.
1872    Not being selfish at all, some people are just...
Name: body, dtype: object